<a href="https://colab.research.google.com/github/timbergrizz/KoBERT_test/blob/main/KoBERT_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 166 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 12.2 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595743 sha256=ff287317a7efeba9fc24b71e8e1d0433ad7bb6b1bed78b4921d15bfcc5c233c1
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 34.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.p

In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-7ndwesdw
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-7ndwesdw
     |████████████████████████████████| 132 kB 15.9 MB/s 
     |████████████████████████████████| 4.5 MB 43.5 MB/s 
     |████████████████████████████████| 9.0 MB 69.5 MB/s 
     |████████████████████████████████| 79 kB 10.4 MB/s 
     |████████████████████████████████| 127 kB 41.3 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15674 sha256=846e65e7dd73046f64cefa09ff564d627e0086d9e02d68675a9a49f08cfc3bc9
  Stored in directory: /tmp/pip-ephem-wheel-cache-wim9huwg/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling ur

In [ ]:
!pip list

Package                       Version
----------------------------- ----------------------------
absl-py                       1.2.0
aiohttp                       3.8.1
aiosignal                     1.2.0
alabaster                     0.7.12
albumentations                1.2.1
altair                        4.2.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arviz                         0.12.1
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
async-timeout                 4.0.2
asynctest                     0.13.0
atari-py                      0.2.9
atomicwrites                  1.4.1
attrs                         22.1.0
audioread                     2.1.9
autograd                      1.4
Babel                         2.10.3
backcall                      0.2.0
beautifulsoup4                4.6.3
bleach                        5.0.1
blis                          0.7.

In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [4]:
#kobert
from kobert.utils import get_tokenizer
from kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
!nvidia-smi

Mon Aug  8 15:32:16 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
#GPU 사용
device = torch.device("cuda:0")

In [7]:
criterion = nn.BCELoss()

In [8]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [53]:
!cp -r /content/drive/MyDrive/colab_files/kobert_test ./

In [15]:
import pandas as pd
sentence_data = pd.read_csv("/content/kobert_test/result.csv")

In [16]:
sentence_data.sample(n = 20)

,sentence,voice_phishing
4199,국민 기업 농협 제외하고요 사장님,True
31065,대출 받을때 필요한 서류가 어떤게 있는지 문자로 보내주실 수 있을까요?,False
20794,"예, 공인인증서가 있어야지 이체를 할 수 있어요.",False
6253,그거를 이렇게 그 출금을 해서,True
14486,그러니까 샷건 해결되셨어요,True
14925,야 그러면 친애저축은행으로 진행을 하시는게 훨씬 빠르실 거예요,True
11927,아니요 못들었어 잘못들었어요,True
18963,해당은행 스마트뱅킹에서 생체인증 서비스 등록후 이용할 수 있습니다.,False
37287,아~네 괜찮은 대출 상품이네요,False
17610,제가 이제 그래프 어떤게 그러니깐 지금 어떤 부분을 말씀하시는거죠 본인께서,True


In [17]:
sentence_data.dtypes

sentence          object
voice_phishing      bool
dtype: object

In [18]:
data_list = []
for q, label in zip(sentence_data["sentence"], sentence_data["voice_phishing"]) :
  d = []
  d.append(q)
  d.append(str(int(label)))

  data_list.append(d)

In [19]:
print(data_list[0])
print(data_list[6000])
print(data_list[12000])
print(data_list[18000])
print(data_list[24000])
print(data_list[30000])
print(data_list[-1])

['이 모든 녹취는 피해자 입증하기 위해 법원에 증거자료로 제출할 예정이고요 네 통화하는 과정에서 주변 소음이나 제 삼 자의 목소리 만 19세 이상 이 목소리가 썼을 경우 법원에 증거자료로 채택 대기가 어렵기 때문에 통화하실 때만큼은 돼 제가 방금 말씀드린 접니다 그 전날 유의 부탁드리겠습니다', '1']
['이쪽으로 고객님 사진찍어서 보내주시면 되세요', '1']
['카페에서 나갈수있는시간', '1']
['신혼부부 전세자금 대출 최저금리가 얼마죠?', '0']
['감사합니다.', '0']
['알겠습니다. 도움 못드려서 죄송합니다.', '0']
['고객센터에 거절한다고 말하라구요?', '0']


In [20]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data_list, test_size = 0.2, random_state = 0)

In [21]:
print(len(train_data), len(test_data))

30400 7600


In [22]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [35]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 50
max_grad_norm = 1
log_interval = 200
learning_rate =  1e-6

In [24]:
#Tokenization
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(train_data, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test_data, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [25]:
data_train[0]

(array([   2, 4299,  517, 7504, 5859, 3271,    3,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(7, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 1)

In [26]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)

In [27]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [28]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [29]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [30]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [31]:
train_dataloader = torch.utils.data.DataLoader(data_train,  batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test,  batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [32]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [33]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [36]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm_notebook(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm_notebook(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/475 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.6790510416030884 train acc 0.578125
epoch 1 batch id 201 loss 0.7195791602134705 train acc 0.5609452736318408
epoch 1 batch id 401 loss 0.6502957940101624 train acc 0.5595776184538653
epoch 1 train acc 0.5606907894736842


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/119 [00:00<?, ?it/s]

...

epoch 49 test acc 0.8762254901960784


  0%|          | 0/475 [00:00<?, ?it/s]

epoch 50 batch id 1 loss 0.4827493131160736 train acc 0.828125
epoch 50 batch id 201 loss 0.4498308002948761 train acc 0.8424284825870647
epoch 50 batch id 401 loss 0.5140653848648071 train acc 0.8429317331670823
epoch 50 train acc 0.843421052631579


  0%|          | 0/119 [00:00<?, ?it/s]

epoch 50 test acc 0.8779324229691876


In [48]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        test_eval=[]
        for i in out:
            logits = i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("일반 문장이")
            elif np.argmax(logits) == 1:
                test_eval.append("보이스피싱이")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [38]:
predict("저는 확인 먼저 해드리곘습니다")

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>> 입력하신 내용에서 보이스피싱이 느껴집니다.


In [39]:
test_data = pd.read_csv("/content/kobert_test/test.csv")

In [43]:
test_result_arr = []
for sentence in test_data["sentence"] :
    test_result_arr.append(predict(sentence))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>> 입력하신 내용에서 보이스피싱이 느껴집니다.
>> 입력하신 내용에서 보이스피싱이 느껴집니다.
>> 입력하신 내용에서 일반 문장이 느껴집니다.
>> 입력하신 내용에서 보이스피싱이 느껴집니다.
>> 입력하신 내용에서 보이스피싱이 느껴집니다.
>> 입력하신 내용에서 보이스피싱이 느껴집니다.
>> 입력하신 내용에서 보이스피싱이 느껴집니다.
>> 입력하신 내용에서 보이스피싱이 느껴집니다.
>> 입력하신 내용에서 보이스피싱이 느껴집니다.
>> 입력하신 내용에서 보이스피싱이 느껴집니다.
>> 입력하신 내용에서 보이스피싱이 느껴집니다.
>> 입력하신 내용에서 보이스피싱이 느껴집니다.
>> 입력하신 내용에서 보이스피싱이 느껴집니다.
>> 입력하신 내용에서 보이스피싱이 느껴집니다.
>> 입력하신 내용에서 보이스피싱이 느껴집니다.
>> 입력하신 내용에서 보이스피싱이 느껴집니다.
>> 입력하신 내용에서 보이스피싱이 느껴집니다.
>> 입력하신 내용에서 보이스피싱이 느껴집니다.
>> 입력하신 내용에서 보이스피싱이 느껴집니다.
>> 입력하신 내용에서 보이스피싱이 느껴집니다.
>> 입력하신 내용에서 보이스피싱이 느껴집니다.
>> 입력하신 내용에서 보이스피싱이 느껴집니다.
>> 입력하신 내용에서 보이스피싱이 느껴집니다.
>> 입력하신 내용에서 보이스피싱이 느껴집니다.
>> 입력하신 내용에서 보이스피싱이 느껴집니다.
>> 입력하신 내용에서 보이스피싱이 느껴집니다.
>> 입력하신 내용에서 일반 문장이 느껴집니다.
>> 입력하신 내용에서 보이스피싱이 느껴집니다.
>> 입력하신 내용에서 보이스피싱이 느껴집니다.
>> 입력하신 내용에서 일반 문장이 느껴집니다.
>> 입력하신 내용에서 보이스피싱이 느껴집니다.
>> 입력하신 내용에서 보이스피싱이 느껴집니다.
>> 입력하신 내용에서 보이스피싱이 느껴집니다.
>> 입력하신 내용에서 보이스피싱이 느껴집니다.
>> 입력하신 내용에서 보이스피싱이 느껴집니다.
>> 입력하신 내용에서 보이스피싱이 느껴집니다.
>> 입력하신 내용에서 보이스피싱이 느껴집니다.
>

In [54]:
test_nornal_data = pd.read_csv("/content/kobert_test/tst_normal.csv")

In [57]:
for i in test_nornal_data["sentence"] :
    print(predict(i))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>> 입력하신 내용에서 일반 문장이 느껴집니다.
None
>> 입력하신 내용에서 일반 문장이 느껴집니다.
None
>> 입력하신 내용에서 일반 문장이 느껴집니다.
None
>> 입력하신 내용에서 일반 문장이 느껴집니다.
None
>> 입력하신 내용에서 일반 문장이 느껴집니다.
None
>> 입력하신 내용에서 일반 문장이 느껴집니다.
None
>> 입력하신 내용에서 일반 문장이 느껴집니다.
None
>> 입력하신 내용에서 일반 문장이 느껴집니다.
None
>> 입력하신 내용에서 일반 문장이 느껴집니다.
None
>> 입력하신 내용에서 일반 문장이 느껴집니다.
None
>> 입력하신 내용에서 일반 문장이 느껴집니다.
None
>> 입력하신 내용에서 일반 문장이 느껴집니다.
None
>> 입력하신 내용에서 일반 문장이 느껴집니다.
None
>> 입력하신 내용에서 일반 문장이 느껴집니다.
None
>> 입력하신 내용에서 일반 문장이 느껴집니다.
None
>> 입력하신 내용에서 일반 문장이 느껴집니다.
None
>> 입력하신 내용에서 일반 문장이 느껴집니다.
None
>> 입력하신 내용에서 일반 문장이 느껴집니다.
None
>> 입력하신 내용에서 일반 문장이 느껴집니다.
None
>> 입력하신 내용에서 일반 문장이 느껴집니다.
None
>> 입력하신 내용에서 일반 문장이 느껴집니다.
None
>> 입력하신 내용에서 일반 문장이 느껴집니다.
None
>> 입력하신 내용에서 일반 문장이 느껴집니다.
None
>> 입력하신 내용에서 일반 문장이 느껴집니다.
None
>> 입력하신 내용에서 일반 문장이 느껴집니다.
None
>> 입력하신 내용에서 일반 문장이 느껴집니다.
None
>> 입력하신 내용에서 일반 문장이 느껴집니다.
None
>> 입력하신 내용에서 일반 문장이 느껴집니다.
None
>> 입력하신 내용에서 일반 문장이 느껴집니다.
None
>> 입력하신 내용에서 일반 문장이 느껴집니다.
None
>> 입력하신 내용에서 일반 문장이 느껴집니다.
None
>> 입력하신 

KeyboardInterrupt: ignored

In [59]:
import torch.onnx 

#Function to Convert to ONNX 
def Convert_ONNX(): 

    # set the model to inference mode 
    model.eval() 

    # Let's create a dummy input tensor  
    dummy_input = torch.randn(1, 8002, requires_grad=True)  

    # Export the model   
    torch.onnx.export(model,         # model being run 
         dummy_input,       # model input (or a tuple for multiple inputs) 
         "voice_phishing_detector.onnx",       # where to save the model  
         export_params=True,  # store the trained parameter weights inside the model file 
         opset_version=10,    # the ONNX version to export the model to 
         do_constant_folding=True,  # whether to execute constant folding for optimization 
         input_names = ['modelInput'],   # the model's input names 
         output_names = ['modelOutput'], # the model's output names 
         dynamic_axes={'modelInput' : {0 : 'batch_size'},    # variable length axes 
                                'modelOutput' : { : 'batch_size'}}) 
    print(" ") 
    print('Model has been converted to ONNX') 

In [62]:
print(model)

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [68]:
torch.save(model.state_dict(), "/content/drive/MyDrive/colab_files/voice_phishing.pth")

In [ ]:
model = BERT